In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

from get_db_url import get_db_url

# Zillow Clustering Exercises

---

## Acquire

### 1

Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [18]:
sql = '''
SELECT
    *
FROM properties_2017
JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
    AND predictions_2017.transactiondate LIKE '2017%%'
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype USING (buildingclasstypeid)
LEFT JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN storytype USING (storytypeid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
    
JOIN (
    SELECT
        parcelid,
        MAX(transactiondate) AS date
    FROM predictions_2017
    GROUP BY parcelid
) AS max_dates ON properties_2017.parcelid = max_dates.parcelid
    AND predictions_2017.transactiondate = max_dates.date
    
WHERE latitude IS NOT NULL AND longitude IS NOT NULL;
'''

zillow = pd.read_sql(sql, get_db_url('zillow'))

In [19]:
zillow = zillow.drop(columns = ['parcelid', 'id', 'date'])

### 2

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [20]:
zillow.shape

(77380, 66)

In [21]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         49439 non-null  float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77380 non-null  float64
 3   buildingclasstypeid           15 non-null     float64
 4   architecturalstyletypeid      206 non-null    float64
 5   airconditioningtypeid         24953 non-null  float64
 6   typeconstructiontypeid        222 non-null    float64
 7   basementsqft                  50 non-null     float64
 8   bathroomcnt                   77380 non-null  float64
 9   bedroomcnt                    77380 non-null  float64
 10  buildingqualitytypeid         49671 non-null  float64
 11  calculatedbathnbr             76771 non-null  float64
 12  decktypeid                    614 non-null    float64
 13  f

In [22]:
zillow.describe().T

,count,mean,std,min,25%,50%,75%,max
heatingorsystemtypeid,49439.0,3.920447e+00,3.592789e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77380.0,2.618264e+02,5.141231e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
typeconstructiontypeid,222.0,6.040541e+00,5.572847e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03
bathroomcnt,77380.0,2.299134e+00,9.966566e-01,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01
bedroomcnt,77380.0,3.053489e+00,1.139103e+00,0.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.600000e+01


In [23]:
dict(zillow.dtypes)

{'heatingorsystemtypeid': dtype('float64'),
 'storytypeid': dtype('float64'),
 'propertylandusetypeid': dtype('float64'),
 'buildingclasstypeid': dtype('float64'),
 'architecturalstyletypeid': dtype('float64'),
 'airconditioningtypeid': dtype('float64'),
 'typeconstructiontypeid': dtype('float64'),
 'basementsqft': dtype('float64'),
 'bathroomcnt': dtype('float64'),
 'bedroomcnt': dtype('float64'),
 'buildingqualitytypeid': dtype('float64'),
 'calculatedbathnbr': dtype('float64'),
 'decktypeid': dtype('float64'),
 'finishedfloor1squarefeet': dtype('float64'),
 'calculatedfinishedsquarefeet': dtype('float64'),
 'finishedsquarefeet12': dtype('float64'),
 'finishedsquarefeet13': dtype('float64'),
 'finishedsquarefeet15': dtype('float64'),
 'finishedsquarefeet50': dtype('float64'),
 'finishedsquarefeet6': dtype('float64'),
 'fips': dtype('float64'),
 'fireplacecnt': dtype('float64'),
 'fullbathcnt': dtype('float64'),
 'garagecarcnt': dtype('float64'),
 'garagetotalsqft': dtype('float64'),


In [24]:
zillow.value_counts()

Series([], dtype: int64)